<a href="https://colab.research.google.com/github/gowthamsaib/solved-problems/blob/main/fcc_sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Question: Neural Network SMS Text Classifier

In this challenge, you need to create a machine learning model that will classify SMS messages as either "ham" or "spam". A "ham" message is a normal message sent by a friend. A "spam" message is an advertisement or a message sent by a company.

You should create a function called predict_message that takes a message string as an argument and returns a list. The first element in the list should be a number between zero and one that indicates the likeliness of "ham" (0) or "spam" (1). The second element in the list should be the word "ham" or "spam", depending on which is most likely.

For this challenge, you will use the SMS Spam Collection dataset. The dataset has already been grouped into train data and test data.

The first two cells import the libraries and data. The final cell tests your model and function. Add your code in between these cells.

In [ ]:
# import libraries
!pip install tensorflow numpy ml-dtypes
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
# Load dataset
train_df = pd.read_csv(train_file_path, sep='\t', header=None, names=["label", "message"])
test_df = pd.read_csv(test_file_path, sep='\t', header=None, names=["label", "message"])

# Encode labels: ham = 0, spam = 1
train_df['label'] = train_df['label'].map({'ham': 0, 'spam': 1})
test_df['label'] = test_df['label'].map({'ham': 0, 'spam': 1})

# Tokenize the text
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(train_df['message'])

X_train = tokenizer.texts_to_sequences(train_df['message'])
X_test = tokenizer.texts_to_sequences(test_df['message'])

# Pad sequences
max_length = 100
X_train = tf.keras.preprocessing.sequence.pad_sequences(X_train, maxlen=max_length, padding='post')
X_test = tf.keras.preprocessing.sequence.pad_sequences(X_test, maxlen=max_length, padding='post')

y_train = np.array(train_df['label'])
y_test = np.array(test_df['label'])

# Build a simple model
model = keras.Sequential([
    keras.layers.Embedding(10000, 64, input_length=max_length),
    keras.layers.Bidirectional(keras.layers.LSTM(32)),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=5, validation_data=(X_test, y_test), verbose=2)




Epoch 1/5
131/131 - 14s - loss: 0.2188 - accuracy: 0.9294 - val_loss: 0.0514 - val_accuracy: 0.9849 - 14s/epoch - 108ms/step
Epoch 2/5
131/131 - 9s - loss: 0.0328 - accuracy: 0.9914 - val_loss: 0.0440 - val_accuracy: 0.9871 - 9s/epoch - 72ms/step
Epoch 3/5
131/131 - 9s - loss: 0.0086 - accuracy: 0.9976 - val_loss: 0.0432 - val_accuracy: 0.9878 - 9s/epoch - 72ms/step
Epoch 4/5
131/131 - 8s - loss: 0.0026 - accuracy: 0.9995 - val_loss: 0.0693 - val_accuracy: 0.9849 - 8s/epoch - 63ms/step
Epoch 5/5
131/131 - 9s - loss: 0.0015 - accuracy: 0.9998 - val_loss: 0.0501 - val_accuracy: 0.9864 - 9s/epoch - 71ms/step


In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
# function to predict messages based on model
def predict_message(pred_text):
  seq = tokenizer.texts_to_sequences([pred_text])
  padded = tf.keras.preprocessing.sequence.pad_sequences(seq, maxlen=max_length, padding='post')
  pred_prob = model.predict(padded)[0][0]
  pred_label = 'spam' if pred_prob > 0.5 else 'ham'
  return [float(pred_prob), pred_label]

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

1/1 [==============================] - 1s 728ms/step
[0.0007029714761301875, 'ham']


In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


1/1 [==============================] - 0s 22ms/step
You passed the challenge. Great job!
